# Running Native Functions

Two of the previous notebooks showed how to [execute semantic functions inline](./03-running-semantic-function-inline.ipynb) and how to [run prompts from a file](./02-running-prompts-from-file.ipynb).

In this notebook, we'll show how to use native functions from a file. We will also show how to call semantic functions from native functions.

This can be useful in a few scenarios:

* Writing logic around how to run a prompt that changes the prompt's outcome.
* Using external data sources to gather data to concatenate into your prompt.
* Validating user input data prior to sending it to the LLM prompt.

Native functions are defined using standard Python code. The structure is simple, but not well documented at this point.

The following examples are intended to help guide new users towards successful native & semantic function use with the Python framework.

Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):

In [ ]:
!python -m pip install -r requirements.txt

Let's create a function that gives us a random number between 3 and a user input as the upper limit. We'll use this number to create 3-x paragraphs of text when passed to a semantic function.

For this example, we will create the following directory structure:

```txt
project-root/
├── __main__.py
└── skills/
    └── GenerateNumber/
        └── native_function.py
    └── WriterSkill/
        └── CorgiStory/
            └── skprompt.txt
            └── config.json
```

First, let's create our native function.
Add the following code to native_function.py

In [ ]:
import random
from semantic_kernel.skill_definition import sk_function

class GenerateNumberSkill:
    """
    Description: Generate a number between 3-x.
    """

    @sk_function(
        description="Generate a random number between 3-x",
        name="GenerateNumber"
    )
    def generate_number(self, input: str) -> str:
        """
        Generate a number between 3-<input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(3, int(input))) 
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

Next, let's create a semantic function that accepts a number as `{{$INPUT}}` and generates that number of paragraphs about two Corgis on an adventure.

Add the following prompt to our `skprompt.txt` file.

```
Write a short story about two Corgis on an adventure.
The storey must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$input}} paragraphs long
```

...and update the config.json file

```
{
    "schema": 1,
    "type": "completion",
    "description": "Generate a story about two Corgis",
    "completion": {
      "max_tokens": 500,
      "temperature": 0.5,
      "top_p": 0.5,
      "presence_penalty": 0.0,
      "frequency_penalty": 0.0
    },
    "default_backends": [
      "text-davinci-003"
    ]
  }
```

Finally, we're ready to setup our script to execute the functions. Open the `__main__.py` file and add the following code.

In [ ]:
import os
import sys
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAITextCompletion, AzureTextCompletion


def main(input):
    # Setup Kernel
    kernel = sk.Kernel()
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.config.add_text_completion_service("text-davinci-003", OpenAITextCompletion("text-davinci-003", api_key, org_id))

    base_skills_directory = os.path.abspath(os.path.join(os.path.dirname(__file__), "./skills/"))
    skills = {
        **kernel.import_native_skill_from_directory(base_skills_directory, "GenerateNumber"),
        **kernel.import_semantic_skill_from_directory(base_skills_directory, "WriterSkill"),
        
    }
    
    # Run the number generator
    number_result = skills["GenerateNumber"].invoke(input=input)

    # Pass the output to the semantic story function
    story = skills["CorgiStory"].invoke(input=number_result.result)

    print(story)

    
if __name__ == "__main__":
    if len(sys.argv) <= 1:
        print("This script requires one input, which should be an integer. Example: `python __main__.py 8`")
    else:
        main(sys.argv[1])

Run the script with `python __main__.py 8` and it will create a story between 3-8 paragraphs.

## Context Variables

Let's expand on our example to add a second input to both the native and the semantic functions.

For the native function, we'll introduce the lower limit variable. This means that a user will input two numbers and the NumberGenerator function will pick a number between the first and second input.

First, let's update our `__main__.py` file to accept the user input and pass the variables to the native and semantic functions. This will leverage the `semantic_kernel.ContextVariables` class.

In [ ]:
import os
import sys
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAITextCompletion, AzureTextCompletion


def main(min: str, max: str, language: str):
    # Setup Kernel
    kernel = sk.Kernel()
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.config.add_text_completion_service("text-davinci-003", OpenAITextCompletion("text-davinci-003", api_key, org_id))

    base_skills_directory = os.path.abspath(os.path.join(os.path.dirname(__file__), "./skills/"))
    skills = {
        **kernel.import_native_skill_from_directory(base_skills_directory, "GenerateNumber"),
        **kernel.import_semantic_skill_from_directory(base_skills_directory, "WriterSkill"),
        
    }

    context_variables = sk.ContextVariables(variables={
        "min": min,
        "max": max,
        "language": language,
        "paragraph_count": ""
    })

    # Run the number generator
    context_variables['paragraph_count'] = skills["GenerateNumber"].invoke(variables=context_variables).result
    
    # Pass the output to the semantic story function
    story = skills["CorgiStory"].invoke(variables=context_variables)

    print(story)

    
if __name__ == "__main__":
    if len(sys.argv) <= 3:
        print("This script requires three inputs. The first two are min, max, which should be integers. The third is the language to write in. Example: `python __main__.py 4 10 english`")
    else:
        main(sys.argv[1], sys.argv[2], sys.argv[3])

Now that we can accept additional inputs from the user and we have those wired up to the functions, we need to update those functions to accept these new arguments.

Let's start with the native function. Notice that we're also adding `@sk_function_context_parameter` decorators to the function here to provide context about what variables need to be provided to the function, and any defaults for those inputs. Using the `@sk_function_context_parameter` decorator provides the name, description and default values for a function's inputs to the [planner.](./05-using-the-planner.ipynb)

In [ ]:
import random
from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter
from semantic_kernel import SKContext

class GenerateNumberSkill:
    """
    Description: Generate a number between 3-x.
    """

    @sk_function(
        description="Generate a random number between min and max",
        name="GenerateNumber"
    )
    @sk_function_context_parameter(name="min", description="Minimum number of paragraphs.")
    @sk_function_context_parameter(name="max", description="Maximum number of paragraphs.", default_value=10)
    def generate_number(self, context: SKContext) -> str:
        """
        Generate a number between min-max
        Example:
            min="4" max="10" => rand(4,8)
        Args:
            min -- The lower limit for the random number generation
            max -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(int(context["min"]), int(context["max"]))) 
        except ValueError as e:
            print(f"Invalid input {context['min']} {context['max']}")
            raise e

Finally, we'll update our semantic function to handle the new inputs.

```
Write a short story about two Corgis on an adventure.
The storey must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
```

Great! We're all set, give it a test: 

```bash
python __main__.py 4 10 english
```

## Calling Semantic Functions from Native Functions with Context Variables

One more example to show here is how we can modify this code to make the native function directly call the semantic function.

Note that in this case, we'd likely want to rename our native function to better represent the intent of producing the story, not just generating a number. However, for simplicity of example, we'll leave the function names as-is.

First, let's trim down our `__main__.py` file to remove the calls to our semantic function.

In [ ]:
import os
import sys
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAITextCompletion, AzureTextCompletion


def main(min: str, max: str, language: str):
    # Setup Kernel
    kernel = sk.Kernel()
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.config.add_text_completion_service("text-davinci-003", OpenAITextCompletion("text-davinci-003", api_key, org_id))

    base_skills_directory = os.path.abspath(os.path.join(os.path.dirname(__file__), "./skills/"))
    skills = {
        **kernel.import_native_skill_from_directory(base_skills_directory, "GenerateNumber"),
        **kernel.import_semantic_skill_from_directory(base_skills_directory, "WriterSkill"),
        
    }

    context_variables = sk.ContextVariables(variables={
        "min": min,
        "max": max,
        "language": language,
        "paragraph_count": ""
    })

    # Pass the output to the semantic story function
    story = skills["GenerateNumber"].invoke(variables=context_variables).result

    print(story)

    
if __name__ == "__main__":
    if len(sys.argv) <= 3:
        print("This script requires three inputs. The first two are min, max, which should be integers. The third is the language to write in. Example: `python __main__.py 4 10 english`")
    else:
        main(sys.argv[1], sys.argv[2], sys.argv[3])

Next, we'll update our native function to call our semantic function and return the value of the semantic function's output.

In [ ]:
import random
from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter
from semantic_kernel import SKContext

class GenerateNumberSkill:
    """
    Description: Generate a number between 3-x.
    """

    @sk_function(
        description="Generate a random number between min and max",
        name="GenerateNumber"
    )
    @sk_function_context_parameter(name="min", description="Minimum number of paragraphs.")
    @sk_function_context_parameter(name="max", description="Maximum number of paragraphs.", default_value=10)
    def generate_number(self, context: SKContext) -> str:
        """
        Generate a number between min-max
        Example:
            min="4" max="10" => rand(4,8)
        Args:
            min -- The lower limit for the random number generation
            max -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            context['paragraph_count'] = str(random.randint(int(context["min"]), int(context["max"]))) 
            return context.skills.get_semantic_function("WriterSkill", "CorgiStory").invoke(variables=context.variables).result
        except ValueError as e:
            print(f"Invalid input {context['min']} {context['max']}")
            raise e

## Calling Native Functions from a prompt/Semantic function

We've now explored how we can call semantic functions from native functions. Next let's do the inverse. 
We will make our CorgiStory semantic function call a native function `GenerateNames` which will return names for our Corgi characters.

For this step we're going to add the new function folder and python file in our directory following this directory structure.

```txt
project-root/
├── __main__.py
└── skills/
    └── GenerateNumber/
        └── native_function.py
    └── GenerateNames/           <====== New Directory
        └── native_function.py  <====== New File
    └── WriterSkill/
        └── CorgiStory/
            └── skprompt.txt
            └── config.json
```

In our `GenerateNames/native_function.py` file we will insert the following code.

In [ ]:
import random
from semantic_kernel.skill_definition import sk_function

class GenerateNamesSkill:
    """
    Description: Generate character names.
    """

    # The default function name will be the name of the function itself, however you can override this
    # by setting the name=<name override> in the @sk_function decorator. In this case, we're using
    # the same name as the function name for simplicity.
    @sk_function(
        description="Generate character names",
        name="generate_names"
    )
    def generate_names(self) -> str:
        """
        Generate two names.
        Returns:
            str
        """
        names = {
            "Hoagie",
            "Hamilton",
            "Bacon",
            "Pizza",
            "Boots",
            "Shorts",
            "Tuna"
        }
        first_name = random.choice(list(names))
        names.remove(first_name)
        second_name = random.choice(list(names))
        return f"{first_name}, {second_name}"

Now we'll add the additional skill to the `__main__.py` function. 

In [ ]:
import os
import sys
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAITextCompletion, AzureTextCompletion


def main(min: str, max: str, language: str):
    # Setup Kernel
    kernel = sk.Kernel()
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.config.add_text_completion_service("text-davinci-003", OpenAITextCompletion("text-davinci-003", api_key, org_id))

    base_skills_directory = os.path.abspath(os.path.join(os.path.dirname(__file__), "./skills/"))
    skills = {
        **kernel.import_native_skill_from_directory(base_skills_directory, "GenerateNumber"),
        **kernel.import_semantic_skill_from_directory(base_skills_directory, "WriterSkill"),

        # Add the following line!
        **kernel.import_native_skill_from_directory(base_skills_directory, "GenerateNames"),
    }

    context_variables = sk.ContextVariables(variables={
        "min": min,
        "max": max,
        "language": language,
        "paragraph_count": ""
    })

    # Run the number generator
    story = skills["GenerateNumber"].invoke(variables=context_variables)

    print(story)

    
if __name__ == "__main__":
    if len(sys.argv) <= 3:
        print("This script requires three inputs. The first two are min, max, which should be integers. The third is the language to write in. Example: `python __main__.py 4 10 english`")
    else:
        main(sys.argv[1], sys.argv[2], sys.argv[3])

Next, let's update our semantic function `CorgiStory` to use our `GenerateName` native function.

```
Write a short story about two Corgis on an adventure.
The storey must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
- The names of the two Corgis are {{ GenerateNames.generate_names }}
```

And now we can run our script again and we'll see that the names of our characters are drawn at random from the list we provided.
This is a simple example of how to us a native function within a semantic function.

### Recap

A quick review of what's happening here:

1. We're loading the kernel with all three of our functions.
2. We're ingesting some variables from the user into our ContextVariables object.
3. We're calling the GenerateNumber native function.
4. The GenerateNumber function sets the value for the number of paragraphs our story should contain, then calls the CorgiStory function, passing in all of the context variables including the paragraph count.
5. The CorgiStory semantic function invokes the native function, GenerateNames.generate_names, which selects two random names for our characters from a predefined list.
6. The CorgiStory semantic function uses the input variables and the output of the generate_names function to build our story.